# 5. Implementation of Scalable Demand Forecasting with PySpark in Google Colab
Similar to setting up Prophet, PySpark installation can be very difficult at times. However, those tasks are extremely easy Google Colaboratory. 

First, go to <a href = "https://research.google.com/colaboratory">Google Colab</a> and click "File" -> "New notebook" to create a new notebook.

### 5.1. Preparation
#### 5.1.1. Mount to Google Drive
For easy access to files, connect the notebook to your Google Drive.

In [1]:
# Import library
from google.colab import drive

# Connect to your google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### 5.1.2. Install PySpark and Prophet
Installing PySpark and Prophet only require one line of code for each.

In [1]:
# Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz

# Unzip
!tar xf spark-3.0.3-bin-hadoop2.7.tgz

# Install spark
!pip install -q findspark

# Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

#
import findspark
findspark.init()

In [ ]:
# Install Prophet                                                                                                                                                                                                  
!pip install Prophet

     |████████████████████████████████| 65 kB 2.4 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
     |████████████████████████████████| 45 kB 2.9 MB/s 
  Created wheel for Prophet: filename=prophet-1.0.1-py3-none-any.whl size=6639062 sha256=76e58c3e0efc63042a223f43034db581b9e0bb262ed89520b73c891df3315c7f
  Stored in directory: /root/.cache/pip/wheels/4e/a0/1a/02c9ec9e3e9de6bdbb3d769d11992a6926889d71567d6b9b67
Successfully built Prophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, but you have cmdstanpy 0.9.68 which is incompatible.


#### 5.1.3. Load necessary packages

In [ ]:
# Import library
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import *
from prophet import Prophet


#### 5.1.4. Upload the CSV file to Google Drive
- Click the folder icon in the left menu as shown in the image below.
- Although you can save anywhere you wish, I like to save it in the Google Drive Colab Notebook folder. To do so, go to "content" -> "dive" -> "MyDrive" -> Colab Notebooks -> create "data" folder
- Click the three dots next to "data". You can upload the CSV file we saved by clicking "Upload"

<img src ="https://github.com/youngdataspace/Time-Series-Forecasting-in-Spark/blob/main/Google%20Colab1.JPG?raw=true">
<img src = "https://github.com/youngdataspace/Time-Series-Forecasting-in-Spark/blob/main/Google%20Colab2.JPG?raw=true">
<img src = "https://github.com/youngdataspace/Time-Series-Forecasting-in-Spark/blob/main/Google%20Colab3.JPG?raw=true">

#### 5.1.5. Import the CSV file and explore it
Import the CSV file we just uploaded to Google Drive.


In [ ]:
# Import the csv file and explore it
sales_pd = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/store_sales.csv')

# Convert ds to datetime
sales_pd['ds'] = pd.to_datetime(sales_pd['ds'])

# Display info
sales_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   ds      913000 non-null  datetime64[ns]
 1   store   913000 non-null  int64         
 2   item    913000 non-null  int64         
 3   y       913000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 27.9 MB


In [ ]:
# Descriptive statistics
sales_pd.describe()

,store,item,y
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.250287
std,2.872283,14.430878,28.801144
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,231.000000


Looks like we correctly have 1-10 stores and 1-50 items.

### 5.2. Prophet x PySpark
#### 5.2.1. Create a Spark session
Spark Sessions utilize Spark's functions. They are created in the Driver program, which is inside the Master node. 

Spark uses Master-Slave architecture. Salve nodes execute the tasks assigned by the Master node.

In [45]:
# Create a Spark Session - Run it on a standalone mode since it is just a practice
# master(): Either yarn or mesos; local[X] when running in standalone
# appName(): Name of the application
# getOrCreate: returns existing SparkSession; otherwise, create a new one
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .getOrCreate()

#### 5.2.2. Structure schema
After reading the CSV file with PySpark we will structure the output of the data. See <a href = "https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.types.StructType.html">here</a> for different types of struct fields.


In [ ]:
# Read the csv file
sales_df = spark.createDataFrame(sales_pd[sales_pd.item==1][sales_pd.store==1])

# Display the schema
sales_df.printSchema()

SyntaxError: ignored

In [ ]:

sales_df.show(5)

In [ ]:
results.createOrReplaceTempView("item_sales")
queried = spark.sql("select * from item_sales")

In [ ]:
# We need to partition all the data based on store for parallel processing
sales_part = (spark.sql(sql).repartition(spark.sparkContext.defaultParallelism, ['store'])).cache()
sales_part.explain()

In [9]:
# Define a schema
schema = StructType([
                     StructField('store', IntegerType()),
                     StructField('item', IntegerType()),
                     StructField('ds', TimestampType()),
                     StructField('y', FloatType()),
                     StructField('yhat', DoubleType()),
                     StructField('yhat_upper', DoubleType()),
                     StructField('yhat_lower', DoubleType()),
                     ])  

#### 5.2.3. Utilize Pandas UDF and PySpark to train multiple models in parallel
The next step is to set parameters, fit the model, and predict sales just as we did for 1 forecast model. We are going to build a function and apply that function to all store-item groups. The only difference between this and our previous 1-model forecast is that we are going to utilize Pandas UDF and PySpark to parallelize the process.

In [10]:
# define the Pandas UDF 
@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def apply_model(store_pd):
  
  # instantiate the model and set parameters
  model = Prophet(
      interval_width=0.95,
      growth='linear',
      daily_seasonality=False,
      weekly_seasonality=True,
      yearly_seasonality=True,
      seasonality_mode='multiplicative'
  )
  
  # fit the model to historical data
  model.fit(store_pd)
  
  # Create a data frame that lists 90 dates starting from Jan 1 2018
  future = model.make_future_dataframe(
      periods=90,
      freq='d',
      include_history=True)
  
  # Out of sample prediction
  future = model.predict(future)

  # Create a data frame that contains store, item, y, and yhat
  f_pd = future[['ds', 'yhat', 'yhat_upper', 'yhat_lower']]
  st_pd = store_pd[['ds', 'store', 'item', 'y']]
  result_pd = f_pd.join(st_pd.set_index('ds'), on='ds', how='left')
  
  # fill store and item
  result_pd['store'] = store_pd['store'].iloc[0]
  result_pd['item'] = store_pd['item'].iloc[0]
  #result_pd['store'] = store_pd['store'].fillna(method='ffill')
  #result_pd['item'] = store_pd['item'].fillna(method='ffill')
  return result_pd[['store', 'item', 'ds', 'y', 'yhat',
                    'yhat_upper', 'yhat_lower']]

In [11]:
# Apply the function to all store-items
results = sales_df.groupby(['store', 'item']).apply(apply_model)

# Print the results - calculate the time to run
import timeit
start = timeit.default_timer()
results.show()
stop = timeit.default_timer()

In [12]:
# Print the time it took to forecast 500 models
print('Time: ', stop - start)   

It only took 29 seconds to train 500 models and forecast 3 months out!

In [13]:
type(results) 

pyspark.sql.dataframe.DataFrame

In [22]:
results.createOrReplaceTempView("item_sales")
queried = spark.sql("select * from item_sales where store==1 AND item==1")

In [26]:
# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [25]:
result_pdf = results.select("*").toPandas()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
sqlContext.sql("SELECT domain_userid, COUNT(*) AS count FROM events GROUP BY domain_userid").show(5)

In [ ]:
final_df = final_df.set_index('ds')
final_df.query('store_id == 25')[['y', 'yhat']].plot()
plt.show()

# 6. Conclusion
In this long post, we went through several topics. We started with identifying trends and seasonality, moved on to building a Prophet model, and scaled the process to model 500 distinct models with PySpark. We didn't get to cover CNN, LSTM, and Seasonal ARIMA but I am planning on adding them in a few days.